In [2]:
import numpy as np
seed = 123
np.random.seed(seed)
import collections
from importlib import reload
import pandas as pd
import sklearn

In [3]:
import xgboost as xgb

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [5]:
import util.data
import util.model
import util.ndcg

# Predict test data

In [11]:
# TODO nrows
# data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';', nrows=10*1000)
# data_test = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';', nrows=10*1000)

data = pd.read_csv('data/training_set_VU_DM_clean.csv', sep=';')
data_test = pd.read_csv('data/test_set_VU_DM_clean.csv', sep=';')

In [15]:
util.data.rm_na(data)
util.data.rm_na(data_test)

k = 'gross_bookings_usd'
gross_booking_scaler = preprocessing.RobustScaler()

data[k] = gross_booking_scaler.fit_transform(data[[k]].values)
data_test.loc[data_test.index, k] = gross_booking_scaler.transform(data_test[[k]].values)

# add noise to reduce dependency on this attr
data.loc[data.index, k] += np.random.normal(loc=0, scale=0.01, size=data[k].size)

# use a single fold, to allow oversampling of bookings or undersampling of clicks/others
folds = util.data.cv_folds_for_sklearn(data, n_cv_folds=1, resampling_ratio=0)
train_indices = folds[0][0]
x_train, y_train = util.data.split_xy(data, selection=train_indices)
x_test = data_test[x_train.columns]

In [16]:
x_train.shape, y_train.shape

((528, 82), (528,))

In [2]:
kwargs = {'random_state': 1234}
# model = xgb.XGBRegressor(criterion='friedman_mse',
#                          n_estimators=100, learning_rate=0.001, max_depth=1,
#                          loss='ls', base_score=0, tol=1e-4, n_jobs=4, **kwargs)
model = xgb.XGBRegressor(objective='rank:ndcg', criterion='friedman_mse', 
                         n_estimators=100, learning_rate=0.001, max_depth=1,
                        loss='ls', base_score=0, tol=1e-4, n_jobs=4, **kwargs)
# model = sklearn.ensemble.RandomForestClassifier(n_jobs=4, n_estimators=500)

model.fit(x_train, y_train)
print(x_train.shape[0], 'samples')

In [20]:
y_pred = model.predict(x_test)
Xy_pred = util.data.Xy_pred(x_test, y_pred, save=True, suffix='xgb-reg-undersampling2')

saved to `data/y_pred_result_xgb-reg-undersampling.csv`


In [39]:
# pd.read_csv('data/y_pred_result_xgb-ndcg.csv', sep=',', nrows=10)

In [25]:
# model = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.01, 
#                           max_depth = 4, alpha = 10, n_estimators = 500, n_jobs=2, seed=123)
# model.fit(x_train, y_train)

In [36]:
pd.read_csv('data/y_pred_result_xgb_1b.csv', sep=',', nrows=10)

,srch_id,prop_id
0,1,54937
1,1,61934
2,1,78599
3,1,99484
4,1,123675
5,1,82231
6,1,94729
7,1,63894
8,1,73666
9,1,28181
